# Beskrivning
I den här notebooken tänkte jag demonstrera parameter-nyckelordet `ref`, som man kan använda sig av i olika situationer. 
> Personligen tycker jag att det är på gränsen till en code smell att använda ref-nyckelordet. I vissa lägen kan det vara bra att använda för att t.ex. optimera minnesanvändning etc.

## Value types
Exemplet nedan visar hur ref-nyckelordet påverkar hur **value types**, värdetyper, vid funktionsanrop. Funktionen `SomeIntFunction` tar in en ref till en integer, som är en värdetyp, den skriver ut värdet före multiplikation med 10 och sedan skriver den ut värdet efteråt.

Nedanför funktionskroppen anropas funktionen med parametern `i = 1` och utskrift av i görs före och efter anropet.

Funktionen returner ett heltal, värdet av p efter multiplikationen. Notera att returen är **värdet** av p och inte referensen. Detta gör att `var res = ...` tilldelas värdet av funktionen, helt skild från inparametern i.

In [ ]:
public int SomeIntFunction(ref int p) 
{
    System.Console.WriteLine("\t(inside 'ref' function) Before multiplication, p = " + p);    
    
    p = p * 10;
    
    System.Console.WriteLine("\t(inside 'ref' function) After multiplication,  p = " + p);

    return p;
}

var i = 1;
System.Console.WriteLine("Before function, i = " + i);
var res = SomeIntFunction(ref i);
System.Console.WriteLine("After function,  i = " + i);

///
System.Console.WriteLine();
System.Console.WriteLine("////////////////////////////////////");
System.Console.WriteLine();
///

System.Console.WriteLine("Before, res = " + res);
res = res * 5;
System.Console.WriteLine("After,  res = " + res);
System.Console.WriteLine("After,    i = " + i);

Before function, i = 1
	(inside 'ref' function) Before multiplication, p = 1
	(inside 'ref' function) After multiplication,  p = 10
After function,  i = 10

////////////////////////////////////

Before, res = 10
After,  res = 50
After,    i = 10


## Reference types, with value type behaviour
I .NET/C# finns det speciella typer, referensetyper, som beter sig som värdertyper. Exempel på detta är `string`. Samma "flöde" som i exemplet ovan visar string-typens värdetypsegenskaper genom att funktionens returvärde tilldelas en ny variabel, `res = ...`, som då INTE är samma instans som inparametern i.

In [ ]:
public string SomeStringFunction(ref string p) 
{
    System.Console.WriteLine("\t(inside 'ref' function) Before add, p = " + p);    
    
    p += " added";
    
    System.Console.WriteLine("\t(inside 'ref' function) After add,  p = " + p);

    return p;
}

var i = "The string";
System.Console.WriteLine("Before function, i = " + i);
var res = SomeStringFunction(ref i);
System.Console.WriteLine("After function,  i = " + i);

///
System.Console.WriteLine();
System.Console.WriteLine("////////////////////////////////////");
System.Console.WriteLine();
///

System.Console.WriteLine("Before, res = " + res);
res += " added-outside-function";
System.Console.WriteLine("After,  res = " + res);
System.Console.WriteLine("After,    i = " + i);

Before function, i = The string
	(inside 'ref' function) Before add, p = The string
	(inside 'ref' function) After add,  p = The string added
After function,  i = The string added

////////////////////////////////////

Before, res = The string added
After,  res = The string added added-outside-function
After,    i = The string added


## Reference types, klasser/objekt
Man kan även använda ref-nyckelordet på parametrar som är referenstyper. Detta kanske känns lite skumt eftersom en sådan paramteter alltid passas in som en referens. ref-nyckelordet får i det här fallet utfallet att funktionen faktiskt helt kan byta ut objektet som referensen pekar på.

Exemplet nedan visar två olika funktioner som tar `Person` respektive `ref Person` som inparameter. Den senare gör alltså att man i tilldelningen `person = new Person(...)` kommer att byta objektet som person-parametern pekar på medan den övre funktionen kommer att allokera ett helt nytt objekt samtidigt som den "yttre referensen" lämnas opåverkad.

In [ ]:
public class Person 
{
    public string Name { get; }

    public Person(string name)
    {
        Name = name;
    }
}

public Person SomePersonFunction(Person person)
{
    System.Console.WriteLine("\t(inside 'plain' function) Before new, person name = " + person.Name);    
    person = new Person("Kalle");
    System.Console.WriteLine("\t(inside 'plain' function) After new,  person name = " + person.Name);  

    return person;
}

public Person SomeOtherPersonFunction(ref Person person)
{
    System.Console.WriteLine("\t(inside 'ref' function) Before new, person name = " + person.Name);    
    person = new Person("Kalle");
    System.Console.WriteLine("\t(inside 'ref' function) After new,  person name = " + person.Name);   

    return person;
}

var p1 = new Person("Nisse");
System.Console.WriteLine("Before 'plain' function,  p1 = " + p1.Name);    

var res1 = SomePersonFunction(p1);

System.Console.WriteLine("After 'plain' function,   p1 = " + p1.Name); 
System.Console.WriteLine("After 'plain' function, res1 = " + res1.Name); 

///
System.Console.WriteLine();
System.Console.WriteLine("////////////////////////////////////");
System.Console.WriteLine();
///

var p2 = new Person("Nisse");
System.Console.WriteLine("Before 'ref' function,  p2 = " + p2.Name);  

var res2 = SomeOtherPersonFunction(ref p2);

System.Console.WriteLine("After 'ref' function,   p2 = " + p2.Name);
System.Console.WriteLine("After 'ref' function, res2 = " + res2.Name);  

Before 'plain' function,  p1 = Nisse
	(inside 'plain' function) Before new, person name = Nisse
	(inside 'plain' function) After new,  person name = Kalle
After 'plain' function,   p1 = Nisse
After 'plain' function, res1 = Kalle

////////////////////////////////////

Before 'ref' function,  p2 = Nisse
	(inside 'ref' function) Before new, person name = Nisse
	(inside 'ref' function) After new,  person name = Kalle
After 'ref' function,   p2 = Kalle
After 'ref' function, res2 = Kalle
